# 1. gradio로 애플, 마이크로소프트, 테슬라, 엔비디아 주식 - yahoo finance에서 2014-전날까지 불러오기
# 2. 데이터 표시
# 3. 종가를 타겟 변수로 지정
# 4. AutoGluon을 이용해 시계열 분석
# 5. 30일 예측 결과 출력, 시각화

In [ ]:
import pandas as pd
import numpy as np
import gradio as gr
import yfinance as yf
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

In [114]:
import yfinance as yf
import pandas as pd
import gradio as gr
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# 종목 리스트
tickers = ["AAPL", "MSFT", "TSLA", "NVDA"]

# 1. Yahoo Finance에서 주식 데이터 불러오기 (종료 날짜를 어제로 설정)
def load_stock_data(ticker):
    end_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    stock_data = yf.download(ticker, start="2014-01-01", end=end_date)
    if stock_data.empty:
        raise ValueError(f"No data available for the selected stock: {ticker}.")
    return stock_data

# 2. 데이터프레임에서 필요한 정보 표시
def display_data(stock_data):
    return stock_data.tail()

# 3. 종가를 타겟 변수로 지정한 후 AutoGluon을 이용한 시계열 분석 수행
def time_series_forecast(stock_data, days=30, time_limit=300):
    stock_data = stock_data.reset_index()
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    
    # AutoGluon에 맞게 열 이름 재정의
    stock_data = stock_data[['Date', 'Close']]
    stock_data.columns = ['datetime', 'target']  # AutoGluon을 위해 열 이름 변경

    # 종목 ID 추가 (AutoGluon에서는 각 시계열 데이터가 고유한 ID를 가져야 함)
    stock_data['item_id'] = 0  # 하나의 시계열만 있으므로 모든 값에 동일한 ID를 부여

    # 데이터를 TimeSeriesDataFrame으로 변환
    ts_data = TimeSeriesDataFrame.from_data_frame(
        stock_data, 
        id_column="item_id", 
        timestamp_column="datetime"
    )

    # 데이터의 빈도를 변환해 일관되게 설정
    ts_data = ts_data.convert_frequency(freq='D')

    # AutoGluon 시계열 예측 모델 초기화 (빈도: 'D' = 일별 데이터)
    predictor = TimeSeriesPredictor(
        prediction_length=days,
        freq='D'  # 일별 빈도 설정
    )
    
    # 학습 수행
    predictor.fit(train_data=ts_data)

    # 30일 미래 예측
    forecast = predictor.predict(ts_data)
    
    return forecast

# 4. 시각화 및 예측 결과 출력
def plot_forecast(stock_data, forecast):
    plt.figure(figsize=(10, 6))
    
    # 과거 데이터 시각화
    plt.plot(stock_data['datetime'], stock_data['target'], label="Actual Prices")
    
    # 예측 데이터 시각화
    plt.plot(forecast.index, forecast['mean'], label="Predicted Prices", linestyle="--")
    
    plt.title("Stock Price Forecast")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True)
    
    # 그래프 저장
    plt.savefig("forecast_plot.png")
    
    return "forecast_plot.png"

# Gradio 인터페이스 구성
def predict_stock(ticker):
    try:
        stock_data = load_stock_data(ticker)
        displayed_data = display_data(stock_data)
        forecast = time_series_forecast(stock_data)
        plot_path = plot_forecast(stock_data, forecast)
    
        return displayed_data, plot_path
    except Exception as e:
        # 오류 발생 시 오류 메시지를 담은 데이터프레임 반환
        error_df = pd.DataFrame({"Error": [str(e)]})
        return error_df, None

# Gradio 앱 UI 정의
interface = gr.Interface(
    fn=predict_stock, 
    inputs=gr.Dropdown(choices=tickers, label="Choose Stock", value="AAPL"),  # 종목 리스트 사용
    outputs=[gr.Dataframe(label="Stock Data"), gr.Image(label="Forecast Plot")],
    title="Stock Price Prediction",
    description="Predict 30-day future prices for Apple, Microsoft, Tesla, or Nvidia stocks using AutoGluon."
)

# 앱 실행
interface.launch()


INFO:httpx:HTTP Request: GET http://127.0.0.1:7865/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7865/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
[*********************100%***********************]  1 of 1 completed
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241018_084332'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Mar 29 23:14:13 UTC 2024
CPU Count:          12
GPU Count:          0
Memory Avail:       4.93 GB / 15.31 GB (32.2%)
Disk Space Avail:   35.88 GB / 237.85 GB (15.1%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'D',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 30,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'ver

In [113]:
interface.close()

Closing server running on port: 7865
